# Dask Bag

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__ 

In [2]:
import dask.bag as db
import json

## Лабораторная работа 12

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.bag` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.bag` к `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.bag` (то есть то, что было получено до вызова `compute`, а не после).

1\. В файлах архива `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива (предварительно разархивировав) в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [3]:
reviews = db.read_text(
    '/Users/nastya/Desktop/Университет/Tobd/12_das_bag/reviews_full/reviews_*.json'
).map(json.loads)

In [5]:
type(reviews)

dask.bag.core.Bag

In [4]:
reviews.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2\. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path` функции `read_text`), использовав для этого регулярное выражение. При создании `bag` разбейте набор данных на сегменты размером 128 Мб (см. аргумент `blocksize`).  Выведите на экран первые 5 элементов полученного `bag`.

In [10]:
import re

In [70]:
def s(review):
    pattern = r'reviews_\d'
    search_pattern = re.findall(pattern, review[1])[0][-1]
    str_for_json = f'{review[0][:-2]}, "rating": {search_pattern}' + '}'
    return json.loads(str_for_json)

In [71]:
review_with_rating = db.read_text(
    '/Users/nastya/Desktop/Университет/Tobd/12_das_bag/reviews_full/reviews_*.json',
    include_path = True,
    blocksize = '128MB'
).map(s)

In [154]:
review_with_rating.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3\. Посчитайте, сколько различных пользователей оставляли отзывы. Выведите это количество на экран.

In [173]:
distinct_review_with_rating = review_with_rating.distinct(key = 'user_id')

In [174]:
len(distinct_review_with_rating.compute())

226570

4\. Создайте версию `bag` из задания 2, в которой сохранены только отзывы, оставленные в 2014 и 2015 годах. Посчитайте и выведите на экран долю, которую составляют отзывы, оставленные в 2014 и 2015 году, среди всех отзывов.

In [12]:
from datetime import datetime

In [73]:
review_with_rating_2014_2015 = review_with_rating.filter(
                            lambda p: datetime.strptime(p['date'], '%Y-%m-%d').year in (2014, 2015)
)

In [187]:
review_with_rating_2014_2015.count().compute()

735274

In [188]:
review_with_rating.count().compute()

9057540

In [189]:
735274/9057540

0.08117811237929945

5\. Для отзывов, оставленных в мае 2013 года, выполните предобработку: 
   * приведите строки к нижнему регистру;
   * обрежьте пробельные символы в начале и конце строки;
   * удалите все символы, кроме английских букв и пробелов.
   
Посчитайте и выведите на экран среднюю длину этих отзывов после предобработки. Если текстовое содержимое отзыва отсутствует, считайте, что длина отзыва равна нулю.

In [74]:
review_with_rating_may_2013 = review_with_rating.filter(
    (lambda p: datetime.strptime(p['date'], '%Y-%m-%d').year == 2013) and 
    (lambda p: datetime.strptime(p['date'], '%Y-%m-%d').month == 5)
)

In [160]:
def processing(data):

    data['review'] = re.sub(r'[^\d a-zA-Z]+', ' ', str(data['review']))
    data['review'] = re.sub(r' +', ' ', data['review'])
    data['review'] = data['review'].strip()
    data['review'] = data['review'].lower()
    
    return data

In [161]:
pros_review_with_rating_may_2013 = review_with_rating_may_2013.map(processing)
pros_review_with_rating_may_2013.take(1)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'wow this is the best i have never been able to make homemade enchiladas that taste like the mexican restaurants i made this last night for my family and they said they will never have enchiladas at the mexican restaurants again thanks for sharing',
  'rating': 0},)

In [165]:
import numpy as np

In [169]:
%%time

mean_pros = np.mean([len(p['review']) for p in pros_review_with_rating_may_2013])

CPU times: user 6.79 s, sys: 7.12 s, total: 13.9 s
Wall time: 3min 24s


In [170]:
mean_pros

268.5630280118798

6\. Посчитайте, как часто встречается та или иная оценка:

   * в 2014 году;
   * в 2015 году.

Создайте `pd.DataFrame`, содержащий 2 столбца: `2014`, `2015` и 6 строк, имеющие индексы строк от 0 до 5. Дайте индексам строк имя `rating`. Выведите полученную таблицу на экран.

In [174]:
review_with_rating_2014 = review_with_rating.filter(
    (lambda p: datetime.strptime(p['date'], '%Y-%m-%d').year == 2014)
)

In [175]:
review_with_rating_2015 = review_with_rating.filter(
    (lambda p: datetime.strptime(p['date'], '%Y-%m-%d').year == 2015)
)

In [177]:
rating_2014 = review_with_rating_2014.pluck('rating').frequencies()

In [178]:
rating_2015 = review_with_rating_2015.pluck('rating').frequencies()

In [180]:
rating_2014.compute()

[(0, 21136), (1, 4344), (2, 4507), (3, 12865), (4, 58433), (5, 258832)]

In [183]:
%%time

rating_2015.compute()

CPU times: user 60.4 ms, sys: 109 ms, total: 169 ms
Wall time: 1min 51s


[(0, 21336), (1, 4902), (2, 4873), (3, 13667), (4, 60980), (5, 269399)]

In [185]:
import pandas as pd

In [214]:
df_rating_2014 = pd.DataFrame(rating_2014.compute())

In [215]:
df_rating_2015 = pd.DataFrame(rating_2015.compute())

In [216]:
df_rating_2014 = df_rating_2014.drop([0], axis = 1).rename({1: 2014}, axis = 1)

In [217]:
df_rating_2015 = df_rating_2015.drop([0], axis = 1).rename({1: 2015}, axis = 1)

In [223]:
df = pd.concat([df_rating_2014, df_rating_2015], axis=1)

In [230]:
df = df.index.rename('rating', inplace = True)

In [232]:
df

,2014,2015
rating,,
0,21136,21336
1,4344,4902
2,4507,4873
3,12865,13667
4,58433,60980
5,258832,269399


7\.  Перераспределите данные из исходного `bag` на $k, k=2, 16, 32, 64, 128$ частей. Для каждого разбиения измерьте время выполнения расчета среднего рейтинга. Визуализируйте зависимость времени выполнения функции от числа сегментов. Временем, затрачиваемым на вызов метода `repartition`, можно пренебречь. Для итерации по $k$ допускается использовать циклы.

In [233]:
review_with_rating_2_part = review_with_rating.repartition(2)
review_with_rating_16_part = review_with_rating.repartition(16)
review_with_rating_32_part = review_with_rating.repartition(32)
review_with_rating_64_part = review_with_rating.repartition(64)
review_with_rating_128_part = review_with_rating.repartition(128)

In [234]:
%%time

review_with_rating_2_part.pluck('rating').mean().compute()

CPU times: user 2min 56s, sys: 9min 9s, total: 12min 6s
Wall time: 39min 25s


4.410805803783367

In [ ]:
%%time

review_with_rating_16_part.pluck('rating').mean().compute()

In [ ]:
%%time

review_with_rating_32_part.pluck('rating').mean().compute()

In [ ]:
%%time

review_with_rating_64_part.pluck('rating').mean().compute()

In [ ]:
%%time

review_with_rating_128_part.pluck('rating').mean().compute()

8\. Будем считать, что сегмент _плохо перемешан_, если в нем _подряд_ идет 5 или более отзывов, оставленных в один и тот же год. Воспользовавшись методом `map_partitions`, посчитайте и выведите на экран, сколько сегментов оказались _плохо перемешанными_ в `bag` из задачи 2. Также выведите на экран общее количество сегментов в `bag`, для которого производились расчеты.

In [244]:
review_with_rating_date = review_with_rating.map(
    lambda p: datetime.strptime(p['date'], '%Y-%m-%d').year
)

In [259]:
lst = [2,4,3,4,7, 7, 7, 7, 6, 7, 7]
bad_mix(lst)

0

In [261]:
def bad_mix(list_of_date):
    
    list_of_date = list(list_of_date)
    
    date_mix = []
    s = 1
    
    for i in range(len(list_of_date) - 1):
        if list_of_date[i] == list_of_date[i+1]:
            s += 1
            date_mix.append(s)
        else:
            s = 1
    return date_mix.count(5)

In [262]:
%%time

review_with_rating_date.map_partitions(bad_mix).compute()

CPU times: user 65.8 ms, sys: 311 ms, total: 377 ms
Wall time: 1min 31s


(0, 1, 0, 0, 1, 3, 4, 3, 1, 1, 3, 4, 1, 1, 1, 1, 0, 0, 5, 2, 2, 1, 2, 1, 0, 1)

In [265]:
sum([0, 1, 0, 0, 1, 3, 4, 3, 1, 1, 3, 4, 1, 1, 1, 1, 0, 0, 5, 2, 2, 1, 2, 1, 0, 1])

39

In [269]:
def bad_mix(list_of_date):
    
    list_of_date = list(list_of_date)
    
    date_mix = []
    s = 1
    
    for i in range(len(list_of_date) - 1):
        if list_of_date[i] == list_of_date[i+1]:
            s += 1
            date_mix.append(s)
            if s == 5:
                return (1, )
        else:
            s = 1
    return (0, )

In [270]:
%%time

review_with_rating_date.map_partitions(bad_mix).sum().compute()

CPU times: user 68.3 ms, sys: 307 ms, total: 376 ms
Wall time: 1min 36s


20

In [266]:
review_with_rating_date.npartitions

26

9\. Используя метод `foldby`, подсчитайте и выведите на экран максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 2.

In [275]:
max_len_review_with_rating = review_with_rating.foldby(
        key = lambda p: p['rating'], #ключ для группировки
        binop = lambda new, p: max(new, len(p['review']) if p['review'] else 0), #текущий и новый - по итогу новый
        initial = 0, 
        combine = lambda one, two: max(one, two) #итог
)

In [276]:
max_len_review_with_rating.compute()

[(0, 6972), (1, 5567), (2, 6972), (3, 4954), (4, 8587), (5, 8587)]